In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

make test input yaml

In [5]:
template = {
    'specimen_id': 'HT206B1-S1H5_U1_07012023_test',
    'ome_tiff': {
        'class': 'File',
        'path': ''
    },
    'group': 'HTAN',
    'project': 'Multiplex_Imaging'
}

In [8]:
yaml.safe_dump(template, open('../tests/data/image_processing_no_thresholds_workflow/inputs.yaml', 'w'))

make cromwell compute1 config

In [9]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/image_processing_no_thresholds_workflow'
output_fp = '../tests/data/image_processing_no_thresholds_workflow/image_processing_no_thresholds_workflow.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

In [10]:
# !mkdir -p ../tests/data/image_processing_no_thresholds_workflow

make execution script

In [11]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/image_processing_no_thresholds_workflow'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'image_processing_no_thresholds_workflow.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'image_processing_no_thresholds_workflow.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']


In [12]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [13]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/image_processing_no_thresholds_workflow/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [14]:
for c in run_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/image_processing_no_thresholds_workflow/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/image_processing_no_thresholds_workflow:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/image_processing_no_thresholds_workflow /storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/image_processing_no_thresholds_workflow:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/image_processing_no_thresholds_workflow /storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"
bgadd -L 10 /estorrs/test_cromwell
export PATH="/opt/java/openjdk/bin:$PATH"
bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=